In [1]:
import re
import pandas as pd
import spacy
import stanza
# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드
nlp = stanza.Pipeline('id', processors='tokenize')


import os
file_list=os.listdir()

from tqdm import tqdm


# 스파이시 영어 모델 로드
nlp = spacy.load('en_core_web_sm')

# 스탄자 인도네시아어 모델 로드
stanza_id = stanza.Pipeline('id', processors='tokenize')

2023-09-27 10:06:15 INFO: Downloading default packages for language: id (Indonesian) ...
2023-09-27 10:06:16 INFO: File exists: C:\Users\USER\stanza_resources\id\default.zip
2023-09-27 10:06:18 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.
2023-09-27 10:06:18 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-09-27 10:06:18 WARNING: Language id package default expects mwt, which has been added
2023-09-27 10:06:18 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-09-27 10:06:18 INFO: Using device: cpu
2023-09-27 10:06:18 INFO: Loading: tokenize
2023-09-27 10:06:18 INFO: Loading: mwt
2023-09-27 10:06:18 INFO: Done loading processors!
2023-09-27 10:06:19 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-09-27 10:06:19 WARNING: Language id package default expects mwt, which has been added
2023-09-27 10:06:19 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-09-27 10:06:19 INFO: Using device: cpu
2023-09-27 10:06:19 INFO: Loading: tokenize
2023-09-27 10:06:19 INFO: Loading: mwt
2023-09-27 10:06:19 INFO: Done loading processors!


</br>
</br>
</br>

# 함수정의
- 불필요 텍스트 제거
- 메타데이터 명시
- 파일 읽기 (데이터 수집 날짜, 키워드 명시)

In [2]:

def cleansing(text):
    # "MMM DD, YYYY" 형식의 날짜와 ")"로 끝나는 부분을 찾아 제거하는 정규표현식
    pattern_1 = r'\w{3}\s\d{2},\s\d{4}\(.*?\)'
    pattern_2 = r'Loading Link to the original story\. Notes.*?sole discretion\.'
    pattern_3 = r'\b[A-Z][a-z]{2} \d{2}, \d{4} \(.*?by Newstex\)'
    
    # 정규표현식을 사용하여 패턴을 제거
    cleaned_text = re.sub(pattern_3, '', text, flags=re.DOTALL)
    
    # cleaned_text = re.sub(pattern_1, '', text)

    # 왼쪽 공백 제거
    cleaned_text = cleaned_text.lstrip()

    if text.endswith('sole discretion.'):
        cleaned_text= cleaned_text[:-1284]
    

    # 날짜 형식 제거 (예: Apr 08, 2013)
    cleaned_text = re.sub(r'\b[A-Z][a-z]{2} \d{2}, \d{4}\b', '', cleaned_text)
    # http:// 또는 https:// 형태 제거
    cleaned_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', cleaned_text)
    # 대괄호 안 숫자 제거
    cleaned_text = re.sub(r'\[\d+\]', '', cleaned_text)
    # <--> 형태 제거
    cleaned_text = re.sub(r'<.*?>', '', cleaned_text)
    

    # 정규표현식 패턴을 사용하여 해당 패턴 제거
    pic_pattern = re.compile(r'pic\.twitter\.com\/[A-Za-z0-9]+')
    cleaned_text = re.sub(pic_pattern, "", cleaned_text)


    # 한글 제거
    text_without_korean = re.sub('[가-힣]+', '', cleaned_text)
    ko_pattern = re.compile(r'[ㄱ-ㅎㅏ-ㅣ]+')
    text_without_korean = re.sub(ko_pattern, '', text_without_korean)
    
    
    # 링크 제거 (http:// 또는 https://로 시작하는 링크)
    text_without_links = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text_without_korean)
    if (text_without_links.startswith("(")) & (")" in text_without_links):
        end_index = text_without_links.index(')')
        text_without_links = text_without_links[end_index+1:]
    if text_without_links.startswith("(KoreanIndo: Delivered by Newstex)"):
        text_without_links = text_without_links[len("(KoreanIndo: Delivered by Newstex)"):]
    
    
    pattern = re.compile(r'(source|Source)')
    match = pattern.search(text_without_links)

    
    if match:
        text_without_links = re.split(r'(source|Source).*', text_without_links)[0].strip()
        
    else:
        pass
    text_without_links = text_without_links.replace("Loading", "")
    
    
    text_without_links = text_without_links.replace("[embedded content]", "")
    text_without_links = re.sub(r'\[ \d+\]:', '', text_without_links)
    return text_without_links


def cleansing_text(test):

    test = re.sub(r"Baca Juga:.*?\.", "", test) #Baca Juga 관련 문장 제거
    test = re.sub(r"Baca Juga :.*?\.", "", test)
    test = re.sub(r"Baca juga:.*?\.", "", test)
    test = re.sub(r"Baca juga.*?\.", "", test)
    test = re.sub(r"Baca juga :.*?\.", "", test)
    test = re.sub(r"baca juga:.*?\.", "", test)
    test = re.sub(r"baca juga :.*?\.", "", test)
    test = re.sub(r"baca Juga:.*?\.", "", test)
    test = re.sub(r"baca Juga :.*?\.", "", test)
    test = re.sub(r"Foto:.*?\.","",test) #Foto 관련 문장 제거
    test = re.sub(r"Written by:.*","",test) #Written by: 이후 모든 문장 제거
    test = re.sub(r"Written by :.*","",test)
    test = re.sub(r"written by:.*","",test)
    test = re.sub(r"written by :.*","",test)
    test = re.sub(r"written by .*","",test)
    test = re.sub(r"—- Written by :.*","",test)
    test = re.sub(r"—Written by :.*","",test)
    test = re.sub(r"\b[A-Z][a-z]{2} \d{2}, \d{4} \(.*?by Newstex\)","",test) #날짜 제거
    test = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+","",test) #링크 제거
    test = re.sub(r"\[\d+\]","",test) #대괄호 포함 내용 제거
    test = re.sub(r"\[ \d+\]","",test) #대괄호 포함 내용 제거
    test = re.sub(r":","",test)
    test = re.sub(r"<.*?>","",test)
    test = re.sub(r"//instagram.com/.*","",test) #//instagram.com/ 이후 모든 문장 제거
    test = re.sub(r"\b(?:\w+\W+){0,30}Soompi(?:\W+\w+){0,30}\b", "", test) #Soompi 기준 앞 뒤 단어 30개 제거
    test = re.sub(r"\b(?:\w+\W+){0,30}soompi(?:\W+\w+){0,30}\b","",test) #soompi
    test = re.sub(r'#\d+;', "", test) ##8216;, #8217; 등 제거
    test = re.sub(r'# \d+;', "", test)
    test = re.sub(r'#\d+ ;', "", test)
    test = re.sub(r'# \d+ ;', "", test)
        
    return test



def second_cleansing_text(dtest):

    dtest = re.sub(r"\[ \d+\]","",dtest)
    dtest = re.sub(r"Written by:.*","",dtest) #Written by: 이후 모든 문장 제거
    dtest = re.sub(r"Written by :.*","",dtest)
    dtest = re.sub(r"written by:.*","",dtest)
    dtest = re.sub(r"written by :.*","",dtest)
    dtest = re.sub(r"//instagram.com/.*","",dtest) #//instagram.com/ 이후 모든 문장 제거
    dtest = re.sub(r"\b(?:\w+\W+){0,30}Soompi(?:\W+\w+){0,30}\b", "", dtest) #Soompi 기준 앞 뒤 단어 30개 제거
    dtest = re.sub(r"\b(?:\w+\W+){0,30}soompi(?:\W+\w+){0,30}\b","",dtest) #soompi
    dtest = re.sub(r":","",dtest)
    dtest = re.sub(r"[一-龥]+","",dtest)

    return dtest

    

# 1 -> 000001, 10231 -> 010231 형태로 id number 바꾸기
def format_id(id_number):
    # Check if the value is NaN
    if pd.isna(id_number):
        return 'NaN'
    
    formatted_id = '{:06d}'.format(int(id_number))
    return formatted_id



#수집일 및 키워드 표기
def set_metadata(df):

    date_str = date

    # YYYY-MM-DD 형태로 수집 날짜 명시
    formatted_date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
    print("collection date :", formatted_date)

    # 파일명에서 확장자(.csv)를 제외한 부분 선택
    prefix = file_name.split('.')[0]
    
    df['Doc_ID'] = prefix + "_" + df['ID'].apply(format_id).astype(str)
    df['Col_Date']= formatted_date
    df['Keyword']=keyword
    
    for idx in df.index :
        if type(df['Date'][idx]) == float:
            # Check if it's the first row
            if idx == df.index[0]:
                # Handle the first row differently. For example, set it to NaN or a default date.
                df['Date'][idx] = pd.NaT
            else:
                df['Date'][idx] = df['Date'][idx-1]



def read_file(file_name):
    global date, keyword
    
    # 파일명에서 날짜와 도시 이름을 추출하는 정규표현식 패턴
    pattern = re.compile(r'(\d{8})_newsdata_(.*)\.csv')
    match = re.match(pattern, file_name)

    if match:
        date = match.group(1)  # 추출한 날짜
        keyword = match.group(2)  # 추출한 도시 이름
        print("Date:", date)
        print("Keyword:", keyword)
    else:
        print("No match found")


    df= pd.read_csv(file_name)
    return df


# 'Article' Column에 있는 기사 본문의 metadata 명시 및 불필요 텍스트 제거 후 'Text' Column으로 생성
def preprocessing(df) :
    set_metadata(df)
    df['Text'] = df['Article'].apply(cleansing_text)
    df['Text'] = df['Text'].apply(cleansing)
    df['Text'] = df['Text'].apply(second_cleansing_text)
    
    

In [3]:
os.getcwd()

'c:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\newsdata_all'

<br>
<br>

## 폴더의 파일들 읽어와서 텍스트 정제 후 데이터프레임 만들기

In [8]:
# 현재 폴더 내의 파일 리스트 얻기
file_list = os.listdir()

# 폴더 내 파일들에 대해 반복하여 CSV 파일 읽기
df_all= pd.DataFrame()
new_df = pd.DataFrame()

for file_name in tqdm(file_list):
    if file_name.endswith('.csv'):
        new_df = read_file(file_name)
        print(f'filename : {file_name}')
        print(f"length : {len(new_df)}")
        preprocessing(new_df)

    df_all =pd.concat([df_all, new_df], axis=0)
print("-----------------------------------\n")

# news_list에 있는 키워드가 들어간 요소들 추리기
news_list= ['KoreanIndo', 'Newstex Blogs', 'The Conversation (Indonesia Edition)', 'Mobile88.com', 'GlobeNewswire Indonesian', 'ParsToday (Indonesian)', 'Antara News', 'Plus Company Updates(PCU)', 'Mongabay.com - Indonesian', 'Thomson Reuters ONE', 'Koreana']
df_all= df_all[df_all['Newspaper'].isin(news_list)]

# 중복 제거 후 날짜 기준으로 정렬
df_all.drop_duplicates(subset='Text', inplace=True)
df_all.sort_values('Date', inplace=True)
df_all.reset_index(drop=True, inplace=True)

df_all

  0%|          | 0/166 [00:00<?, ?it/s]

Date: 20230808
Keyword: Korea
filename : 20230808_newsdata_Korea.csv
length : 7454
collection date : 2023-08-08


C:\Users\USER\AppData\Local\Temp\ipykernel_17356\83953182.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'][idx] = df['Date'][idx-1]
  1%|          | 2/166 [01:04<1:28:33, 32.40s/it]

Date: 20230809
Keyword: Blackpink
filename : 20230809_newsdata_Blackpink.csv
length : 884
collection date : 2023-08-09


  2%|▏         | 3/166 [01:09<56:31, 20.80s/it]  

...

 94%|█████████▍| 156/166 [17:30<01:36,  9.62s/it]

Date: 20230920
Keyword: Koreana2
filename : 20230920_newsdata_Koreana2.csv
length : 270
collection date : 2023-09-20


100%|██████████| 166/166 [17:42<00:00,  6.40s/it]


-----------------------------------



,ID,Source_File,Newspaper,Date,Length,Section,Author,Edition,Headline,Graphic,Article,Doc_ID,Col_Date,Keyword,Text
0,11,C:/Users/USER/Desktop/news data/20230920_data/...,Koreana,2012-07-01,0,NaN,Soul Ho-Jeong,NaN,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,False,"Pada suatu ketika, saat hanya istana kerajaan ...",20230920_newsdata_Koreana1_000011,2023-09-20,Koreana1,"Pada suatu ketika, saat hanya istana kerajaan ..."
1,10,C:/Users/USER/Desktop/news data/20230920_data/...,Koreana,2012-07-01,0,NaN,Koh Mi-seok,NaN,"Yang Hague, Seniman Visual yang Melampaui Genr...",False,Yang Haegue (Yang Hye-gyu) tengah menikmati pe...,20230920_newsdata_Koreana1_000010,2023-09-20,Koreana1,Yang Haegue (Yang Hye-gyu) tengah menikmati pe...
2,19,C:/Users/USER/Desktop/news data/20230920_data/...,Koreana,2012-07-01,0,NaN,Lee Kwang-pyo,NaN,Warisan Budaya Tak-Benda Abad 21\n\nKekayaan B...,False,Restorasi Sungnyemun merupakan proyek konstruk...,20230920_newsdata_Koreana1_000019,2023-09-20,Koreana1,Restorasi Sungnyemun merupakan proyek konstruk...
3,18,C:/Users/USER/Desktop/news data/20230920_data/...,Koreana,2012-07-01,0,NaN,Song Hye-jin,NaN,Warisan Budaya-Tak Benda pada Abad 21\n\nDari ...,False,Program UNESCO tentang pendaftaran warisan bud...,20230920_newsdata_Koreana1_000018,2023-09-20,Koreana1,Program UNESCO tentang pendaftaran warisan bud...
4,17,C:/Users/USER/Desktop/news data/20230920_data/...,Koreana,2012-07-01,0,NaN,Ye Jong-suk,NaN,Tradisi Kuliner\nApa dan Bagaimana Cara Melest...,False,Tradisi budaya apa yang harus kita pertahankan...,20230920_newsdata_Koreana1_000017,2023-09-20,Koreana1,Tradisi budaya apa yang harus kita pertahankan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46095,27,C:/Users/USER/Desktop/news data/20230809_data/...,Newstex Blogs,2023-05-09,208 words,NaN,Kidihae,NaN,NewJeans Terdaftar di 'Guinness Book of World ...,False,"May 09, 2023( KoreanIndo: http://koreanindo.ne...",20230809_newsdata_Kpop_000027,2023-08-09,Kpop,"Menurut ADOR pada 9 Mei KST, NewJeans terda..."
46096,8898,C:/Users/USER/Desktop/news data/20230808_data_...,ParsToday (Indonesian),2023-08-03,417 words,NaN,Pars Today,NaN,RI Optimistis ASEAN Bisa Rawat Perdamaian Regi...,False,Menteri Pertahanan RI Prabowo Subianto optimis...,20230808_newsdata_Korea_008898,2023-08-08,Korea,Menteri Pertahanan RI Prabowo Subianto optimis...
46097,8899,C:/Users/USER/Desktop/news data/20230808_data_...,Newstex Blogs,2023-08-03,462 words,NaN,anisrina,NaN,"Seok Matthew & Kim Tae Rae Positif COVID-19, Z...",False,"Aug 02, 2023( KoreanIndo: http://koreanindo.ne...",20230808_newsdata_Korea_008899,2023-08-08,Korea,ZEROBASEONE membatalkan dan menunda sejumla...
46098,8900,C:/Users/USER/Desktop/news data/20230808_data_...,Newstex Blogs,2023-08-03,217 words,NaN,Kidihae,NaN,Agensi Lee Do Hyun Mengonfirmasi Tanggal Penda...,False,"Aug 03, 2023( KoreanIndo: http://koreanindo.ne...",20230808_newsdata_Korea_008900,2023-08-08,Korea,"Pada 3 Agustus, iMBC mengabarkan bahwa Lee ..."


<br>
<br>
<br>

# 문장 분리

In [9]:
def tokenizing(df):
    new_rows = []
    max_length = 5
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        article_text = row['Text']
        doc = nlp(article_text)
        sen_id = 1  # 문장 ID를 1로 초기화


        for sentence in doc.sents:  # spacy에서는 .sents로 문장에 접근
            sentence = re.sub(r'shared from .*', "", sentence.text)
            sentence = re.sub(r'Shared from .*', "", sentence)
            sentence = re.sub(r'shared by .*', "", sentence)
            sentence = re.sub(r'Shared by .*', "", sentence)
            spacy_doc = nlp(sentence)  # Renamed to spacy_doc for clarity
            tokens = [token.text for sent in spacy_doc.sents for token in sent]
            
            # 문장, 토큰, 문장길이, Sen_ID 컬럼 추가
            new_row = row.copy()
            new_row['Sentence'] = sentence
            new_row['Token'] = tokens
            new_row['Sen_len'] = len(tokens)
            new_row['Sen_ID'] = f"{row['Doc_ID']}_sen{sen_id:06d}"
            
            new_rows.append(new_row)
            sen_id += 1  # 문장 ID 증가


    # 리스트 기호 없이 단어 단위로 토큰화
    new_df = pd.DataFrame(new_rows)
    new_df['Pub_Type']='Newspaper'
    new_df['Tokenized_Sentence']= new_df['Token'].apply(' '.join)

    # 'Sentence' 컬럼 중심으로 중복 제거
    new_df.drop_duplicates(subset='Sentence', inplace=True)
    new_df.reset_index(drop=True, inplace=True)

    # 공식 제출용 컬럼명으로 컬럼명 변환
    new_df.rename(columns={'Source_File' : 'Filename', 'Headline':'Title', 'Keyword':'Pub_Subj', 'Date': 'Pub_Date', 'Sen_len': 'Word_Count'}, inplace=True)
    
    new_df = new_df[['Doc_ID', 'Filename', 'Title', 'Pub_Type', 'Pub_Subj', 'Pub_Date', 'Col_Date', 'Sen_ID', 'Word_Count', 'Text', 'Sentence','Tokenized_Sentence' , 'Token']]
    new_df = new_df[new_df['Token'].apply(lambda x: len(x) > max_length)]
    return new_df


<br>
<br>

### 토큰화 완료된 데이터프레임

In [10]:
# 토큰화 완료된 최종 데이터프레임

df = tokenizing(df_all)
df

Processing rows: 100%|██████████| 46100/46100 [2:32:33<00:00,  5.04it/s]   


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000001,32,"Pada suatu ketika, saat hanya istana kerajaan ...","Pada suatu ketika, saat hanya istana kerajaan ...","Pada suatu ketika , saat hanya istana kerajaan...","[Pada, suatu, ketika, ,, saat, hanya, istana, ..."
1,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000002,28,"Pada suatu ketika, saat hanya istana kerajaan ...",Dedikasi mereka telah membantu keberlangsungan...,Dedikasi mereka telah membantu keberlangsungan...,"[Dedikasi, mereka, telah, membantu, keberlangs..."
2,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000003,35,"Pada suatu ketika, saat hanya istana kerajaan ...","Bahkan, setelah dia akhirnya meyakinkan mereka...","Bahkan , setelah dia akhirnya meyakinkan merek...","[Bahkan, ,, setelah, dia, akhirnya, meyakinkan..."
3,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000004,23,"Pada suatu ketika, saat hanya istana kerajaan ...",Mereka sangat khawatir kepada diskriminasi yan...,Mereka sangat khawatir kepada diskriminasi yan...,"[Mereka, sangat, khawatir, kepada, diskriminas..."
4,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000005,23,"Pada suatu ketika, saat hanya istana kerajaan ...","Pada 10 Juli 1960, tulisan pertama dari serial...","Pada 10 Juli 1960 , tulisan pertama dari seria...","[Pada, 10, Juli, 1960, ,, tulisan, pertama, da..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
518844,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000196,6,"Syarifah Fadiyah Alkaff, remaja yang masih ...",Di bawah pohon mangga?',Di bawah pohon mangga ? ',"[Di, bawah, pohon, mangga, ?, ']"
518846,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000198,17,"Syarifah Fadiyah Alkaff, remaja yang masih ...","'Saya nggak mau jual, ini satu-satunya harta k...","' Saya nggak mau jual , ini satu - satunya har...","[', Saya, nggak, mau, jual, ,, ini, satu, -, s..."
518847,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000199,13,"Syarifah Fadiyah Alkaff, remaja yang masih ...","Nggak tahu nanti kalau saya sudah mati,' kata ...","Nggak tahu nanti kalau saya sudah mati , ' kat...","[Nggak, tahu, nanti, kalau, saya, sudah, mati,..."
518848,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000200,7,"Syarifah Fadiyah Alkaff, remaja yang masih ...","Hafsah, usianya kini 90 tahun.","Hafsah , usianya kini 90 tahun .","[Hafsah, ,, usianya, kini, 90, tahun, .]"


In [11]:
# 날짜_newsdata_원천데이터 합본.pickle

df.to_pickle('20230926_news_assemble_unique.pickle')

In [12]:
import pandas as pd
import re

words_to_remove = ['/instagram', 'Written by', 'written by', 'WRITTEN BY', 'POWERED BY',
                    'ads', 'full credits', 'Full Credits', 'FULL CREDITS']

# Function to remove sentences containing specified words
def remove_sentences_with_words(text, words):
    # Split the text into sentences using a simple regex
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Initialize a list to store the filtered sentences
    filtered_sentences = []

    for sentence in sentences:
        # Check if any of the specified words exist in the sentence (case insensitive)
        if any(re.search(fr'\b{re.escape(word)}\b', sentence, re.IGNORECASE) for word in words):
            continue  # Skip this sentence if it contains any of the specified words
        else:
            filtered_sentences.append(sentence)

    # Join the filtered sentences back into a single text
    result = ' '.join(filtered_sentences)
    return result



In [13]:
# 데이터프레임에 적용
import numpy as np


df['Tokenized_Sentence'] = df['Tokenized_Sentence'].apply(remove_sentences_with_words, words=words_to_remove)

df['Tokenized_Sentence'].replace('', np.nan, inplace=True)
df = df.dropna()

df


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000001,32,"Pada suatu ketika, saat hanya istana kerajaan ...","Pada suatu ketika, saat hanya istana kerajaan ...","Pada suatu ketika , saat hanya istana kerajaan...","[Pada, suatu, ketika, ,, saat, hanya, istana, ..."
1,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000002,28,"Pada suatu ketika, saat hanya istana kerajaan ...",Dedikasi mereka telah membantu keberlangsungan...,Dedikasi mereka telah membantu keberlangsungan...,"[Dedikasi, mereka, telah, membantu, keberlangs..."
2,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000003,35,"Pada suatu ketika, saat hanya istana kerajaan ...","Bahkan, setelah dia akhirnya meyakinkan mereka...","Bahkan , setelah dia akhirnya meyakinkan merek...","[Bahkan, ,, setelah, dia, akhirnya, meyakinkan..."
3,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000004,23,"Pada suatu ketika, saat hanya istana kerajaan ...",Mereka sangat khawatir kepada diskriminasi yan...,Mereka sangat khawatir kepada diskriminasi yan...,"[Mereka, sangat, khawatir, kepada, diskriminas..."
4,20230920_newsdata_Koreana1_000011,C:/Users/USER/Desktop/news data/20230920_data/...,Warisan Budaya Tak-Benda di Abad 21\nSahabat G...,Newspaper,Koreana1,2012-07-01,2023-09-20,20230920_newsdata_Koreana1_000011_sen000005,23,"Pada suatu ketika, saat hanya istana kerajaan ...","Pada 10 Juli 1960, tulisan pertama dari serial...","Pada 10 Juli 1960 , tulisan pertama dari seria...","[Pada, 10, Juli, 1960, ,, tulisan, pertama, da..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
518844,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000196,6,"Syarifah Fadiyah Alkaff, remaja yang masih ...",Di bawah pohon mangga?',Di bawah pohon mangga ? ',"[Di, bawah, pohon, mangga, ?, ']"
518846,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000198,17,"Syarifah Fadiyah Alkaff, remaja yang masih ...","'Saya nggak mau jual, ini satu-satunya harta k...","' Saya nggak mau jual , ini satu - satunya har...","[', Saya, nggak, mau, jual, ,, ini, satu, -, s..."
518847,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000199,13,"Syarifah Fadiyah Alkaff, remaja yang masih ...","Nggak tahu nanti kalau saya sudah mati,' kata ...","Nggak tahu nanti kalau saya sudah mati , ' kat...","[Nggak, tahu, nanti, kalau, saya, sudah, mati,..."
518848,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000200,7,"Syarifah Fadiyah Alkaff, remaja yang masih ...","Hafsah, usianya kini 90 tahun.","Hafsah , usianya kini 90 tahun .","[Hafsah, ,, usianya, kini, 90, tahun, .]"


In [14]:
# 문장 수 확인
df['Word_Count'].sum()

10125771

In [29]:
# 토큰별 작업용 컬럼 생성
for i in range(df['Word_Count'].max()):
    new_column_name = f'Column_{i}'  # 새 열 이름 생성
    df[new_column_name]=np.nan

for idx in tqdm(df.index):
    for i, token in enumerate(df['Token'][idx]):
        new_column_name = f'Column_{i}'  # 새 열 이름 생성
        df[new_column_name][idx]=token

C:\Users\USER\AppData\Local\Temp\ipykernel_16016\2584547734.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name]=np.nan
C:\Users\USER\AppData\Local\Temp\ipykernel_16016\2584547734.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_name]=np.nan
C:\Users\USER\AppData\Local\Temp\ipykernel_16016\2584547734.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [15]:
# 'Sen_ID'기준으로 정렬

df.sort_values('Sen_ID', inplace=True)
df.reset_index(drop=True, inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_17356\81318995.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values('Sen_ID', inplace=True)


In [16]:
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,22,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Kyuhyun Super Junior , Suho EXO , dan Lee Se...","[ , Kyuhyun, Super, Junior, ,, Suho, EXO, ,, d..."
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,15,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",Who Laughs' tayang perdana September lalu di K...,Who Laughs ' tayang perdana September lalu di ...,"[Who, Laughs, ', tayang, perdana, September, l..."
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Sebagai hasil dari keberhasilannya, tayangan u...","Sebagai hasil dari keberhasilannya , tayangan ...","[Sebagai, hasil, dari, keberhasilannya, ,, tay..."
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Berdasarkan novel karya Victor Hugo, 'The Man ...","Berdasarkan novel karya Victor Hugo , ' The Ma...","[Berdasarkan, novel, karya, Victor, Hugo, ,, '..."
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",Musikal ini mengkritik jatuhnya keadilan dan k...,Musikal ini mengkritik jatuhnya keadilan dan k...,"[Musikal, ini, mengkritik, jatuhnya, keadilan,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494292,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000072,18,Trashbusters tidak putus asa saat memandang tu...,Era yang memungkinkan apa saja dapat diantar m...,Era yang memungkinkan apa saja dapat diantar m...,"[Era, yang, memungkinkan, apa, saja, dapat, di..."
494293,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000073,23,Trashbusters tidak putus asa saat memandang tu...,Meski penggunaan kantong plastik sudah dilaran...,Meski penggunaan kantong plastik sudah dilaran...,"[Meski, penggunaan, kantong, plastik, sudah, d..."
494294,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000074,10,Trashbusters tidak putus asa saat memandang tu...,Hal-hal itu mungkin dapat memicu tawa sinis.,Hal - hal itu mungkin dapat memicu tawa sinis .,"[Hal, -, hal, itu, mungkin, dapat, memicu, taw..."
494295,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000075,9,Trashbusters tidak putus asa saat memandang tu...,"Meskipun demikian, mari kita mengucapkan sloga...","Meskipun demikian , mari kita mengucapkan slog...","[Meskipun, demikian, ,, mari, kita, mengucapka..."


In [17]:
len(df)

494297

<br>

### 합쳐진 원본데이터(보관용) pickle 형태로 만들기

In [18]:
# 날짜_newsdata_원천데이터 합본.pickle

df.to_pickle('20230926_news_assemble_unique.pickle')

In [35]:
df=df.to_csv('20230922_news_assemble_unique.csv')

In [36]:
os.getcwd()

'c:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\newsdata_all'

In [33]:
# 총 수집한 토큰의 갯수 출력

df['Word_Count'].sum()

10124694

<br>
<br>

## 날짜 단위로 기존 데이터 제거

In [8]:
## target_date 수정 후 사용

target_date='2023-08-23' 
df_new = df[df['Col_Date'] >= target_date]
df_new.sort_values('Sen_ID', inplace=True)
df_new.reset_index(drop=True, inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_19300\189799062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0822.sort_values('Sen_ID', inplace=True)


<br>
<br>

# 가공된 데이터 제거

In [52]:
#20230904 조각 나누기 전 이미 가공된 파일 제외
df_finish = pd.read_excel('C:\\Users\\Juhee Kim\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\원천데이터\\가공완료데이터\\테스트웍스_인도네시아어 현황 관리 파일_230906.xlsx')
df_finish

,원천 전달 날짜,원천 파일명,가공 완료 날짜,주관기관 전달 파일,비고,파일명,Doc_ID,Pub_Type,Pub_Subj,Sen_ID,Word_Count
0,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,1,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000001,29
1,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,0,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000002,26
2,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,0,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000003,8
3,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,0,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000004,27
4,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-08-29,초기데이터가공_제출본_0829_TW (version 1) - 공유본,NaN,0,20230808_newsdata_Korea_001000,Newspaper,Korea,20230808_newsdata_Korea_001000_000005,11
...,...,...,...,...,...,...,...,...,...,...,...
9582,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-09-04,NaN,NaN,0,20230808_newsdata_Korea_002210,Newspaper,Korea,20230808_newsdata_Korea_002210_000019,30
9583,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-09-04,NaN,NaN,0,20230808_newsdata_Korea_002210,Newspaper,Korea,20230808_newsdata_Korea_002210_000020,14
9584,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-09-04,NaN,NaN,0,20230808_newsdata_Korea_002210,Newspaper,Korea,20230808_newsdata_Korea_002210_000021,30
9585,2023-08-21,20230821_원천데이터(기사)_1.csv,2023-09-04,NaN,NaN,0,20230808_newsdata_Korea_002210,Newspaper,Korea,20230808_newsdata_Korea_002210_000022,42


In [53]:
# 가공된 데이터에서 'Doc_ID' 리스트로 추출

df_filtered = df_finish['Doc_ID'].tolist()
df_filtered

['20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001000',
 '20230808_newsdata_Korea_001001',
 '20230808_newsdata_Korea_001001',
 '20230808_newsdata_Korea_001001',
 '20230808_newsdata_Korea_001001',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001002',
 '20230808_newsdata_Korea_001003',
 '20230808_newsdata_Korea_001003',
 '20230808_newsdata_Korea_001004',
 '20230808_newsdata_Korea_001004',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_Korea_001005',
 '20230808_newsdata_

In [46]:
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,...,Column_237,Column_238,Column_239,Column_240,Column_241,Column_242,Column_243,Column_244,Column_245,Column_246
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,23,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,15,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392852,20230905_newsdata_Yeon_002695,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230905_dat...,Park Si Yeon Melahirkan Bayi Perempuan,Newspaper,Yeon,2013-09-24,2023-09-05,20230905_newsdata_Yeon_002695_sen000001,23,"Park Si Yeon telah melahirkan bayi perempuan, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392853,20230905_newsdata_Yeon_002695,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230905_dat...,Park Si Yeon Melahirkan Bayi Perempuan,Newspaper,Yeon,2013-09-24,2023-09-05,20230905_newsdata_Yeon_002695_sen000002,25,"Park Si Yeon telah melahirkan bayi perempuan, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392854,20230905_newsdata_Yeon_002695,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230905_dat...,Park Si Yeon Melahirkan Bayi Perempuan,Newspaper,Yeon,2013-09-24,2023-09-05,20230905_newsdata_Yeon_002695_sen000003,7,"Park Si Yeon telah melahirkan bayi perempuan, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392855,20230905_newsdata_Yeon_002695,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230905_dat...,Park Si Yeon Melahirkan Bayi Perempuan,Newspaper,Yeon,2013-09-24,2023-09-05,20230905_newsdata_Yeon_002695_sen000004,14,"Park Si Yeon telah melahirkan bayi perempuan, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# 'Doc_ID'기준으로 가공된 데이터 제외

df_final = df[~df['Doc_ID'].isin(df_filtered)]
df_final

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,...,Column_840,Column_841,Column_842,Column_843,Column_844,Column_845,Column_846,Column_847,Column_848,Column_849
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,38,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,18,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346847,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000004,7,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346848,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000005,15,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346849,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000006,6,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346850,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000007,10,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#20230922 Doc_ID 기준 중복 기사 제거
import pandas as pd
import os
path = 'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\원천데이터\\기사데이터\\20230911\\20230911-1'
os.chdir(path)
os.getcwd()

'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\원천데이터\\기사데이터\\20230911\\20230911-1'

In [20]:
df = pd.read_pickle('C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\newsdata_all\\20230926_news_assemble_unique.pickle')
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,22,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Kyuhyun Super Junior , Suho EXO , dan Lee Se...","[ , Kyuhyun, Super, Junior, ,, Suho, EXO, ,, d..."
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,15,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",Who Laughs' tayang perdana September lalu di K...,Who Laughs ' tayang perdana September lalu di ...,"[Who, Laughs, ', tayang, perdana, September, l..."
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Sebagai hasil dari keberhasilannya, tayangan u...","Sebagai hasil dari keberhasilannya , tayangan ...","[Sebagai, hasil, dari, keberhasilannya, ,, tay..."
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...","Berdasarkan novel karya Victor Hugo, 'The Man ...","Berdasarkan novel karya Victor Hugo , ' The Ma...","[Berdasarkan, novel, karya, Victor, Hugo, ,, '..."
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",Musikal ini mengkritik jatuhnya keadilan dan k...,Musikal ini mengkritik jatuhnya keadilan dan k...,"[Musikal, ini, mengkritik, jatuhnya, keadilan,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494292,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000072,18,Trashbusters tidak putus asa saat memandang tu...,Era yang memungkinkan apa saja dapat diantar m...,Era yang memungkinkan apa saja dapat diantar m...,"[Era, yang, memungkinkan, apa, saja, dapat, di..."
494293,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000073,23,Trashbusters tidak putus asa saat memandang tu...,Meski penggunaan kantong plastik sudah dilaran...,Meski penggunaan kantong plastik sudah dilaran...,"[Meski, penggunaan, kantong, plastik, sudah, d..."
494294,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000074,10,Trashbusters tidak putus asa saat memandang tu...,Hal-hal itu mungkin dapat memicu tawa sinis.,Hal - hal itu mungkin dapat memicu tawa sinis .,"[Hal, -, hal, itu, mungkin, dapat, memicu, taw..."
494295,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000075,9,Trashbusters tidak putus asa saat memandang tu...,"Meskipun demikian, mari kita mengucapkan sloga...","Meskipun demikian , mari kita mengucapkan slog...","[Meskipun, demikian, ,, mari, kita, mengucapka..."


In [21]:
df_1 = pd.read_csv('202300911_원천데이터(기사)_1.csv')
df_2 = pd.read_csv('202300911_원천데이터(기사)_2.csv')
df_3 = pd.read_csv('202300911_원천데이터(기사)_3.csv')
df_4 = pd.read_csv('202300911_원천데이터(기사)_4.csv')
df_5 = pd.read_csv('202300911_원천데이터(기사)_5.csv')
df_6 = pd.read_csv('202300911_원천데이터(기사)_6.csv')
df_7 = pd.read_csv('202300911_원천데이터(기사)_7.csv')


C:\Users\USER\AppData\Local\Temp\ipykernel_17356\4264928424.py:1: DtypeWarning: Columns (98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,3

In [22]:
df_combined = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7], ignore_index=True)
df_combined

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,...,Column_840,Column_841,Column_842,Column_843,Column_844,Column_845,Column_846,Column_847,Column_848,Column_849
0,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000001,38,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000002,13,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000003,33,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000004,17,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230808_newsdata_Korea_000001,C:/Users/USER/Desktop/news data/20230808_data_...,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok H...",Newspaper,Korea,2019-10-30,2023-08-08,20230808_newsdata_Korea_000001_sen000005,18,"Kyuhyun Super Junior, Suho EXO, dan Lee Seok ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337688,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000004,7,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337689,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000005,15,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337690,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000006,6,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337691,20230908_newsdata_Myung_000599,C:/Users/USER/Desktop/기사 데이터_로컬환경/20230908_dat...,Gong Myung Cerita Tentang Perannya Sebagai Dew...,Newspaper,Myung,2017-04-09,2023-09-08,20230908_newsdata_Myung_000599_sen000007,10,Dalam wawancara untuk edisi April dari majala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
unique_doc_ids = df[~df['Doc_ID'].isin(df_combined['Doc_ID'])][['Doc_ID']].reset_index(drop=True)
unique_doc_ids

,Doc_ID
0,20230808_newsdata_Korea_000400
1,20230808_newsdata_Korea_000400
2,20230808_newsdata_Korea_000400
3,20230808_newsdata_Korea_000400
4,20230808_newsdata_Korea_000400
...,...
103423,20230920_newsdata_Koreana2_000270
103424,20230920_newsdata_Koreana2_000270
103425,20230920_newsdata_Koreana2_000270
103426,20230920_newsdata_Koreana2_000270


In [24]:
unique_df = df[~df['Doc_ID'].isin(df_combined['Doc_ID'])].reset_index(drop=True)
unique_df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000001,27,"Saat ini, banyak kafe-kafe baru yang berlomba ...","Saat ini, banyak kafe-kafe baru yang berlomba ...","Saat ini , banyak kafe - kafe baru yang berlom...","[Saat, ini, ,, banyak, kafe, -, kafe, baru, ya..."
1,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000002,28,"Saat ini, banyak kafe-kafe baru yang berlomba ...",Dengan hanya sebulan setelah dibuka di daerah ...,Dengan hanya sebulan setelah dibuka di daerah ...,"[Dengan, hanya, sebulan, setelah, dibuka, di, ..."
2,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000003,31,"Saat ini, banyak kafe-kafe baru yang berlomba ...",Para penggemar drama Korea pasti akan teringat...,Para penggemar drama Korea pasti akan teringat...,"[Para, penggemar, drama, Korea, pasti, akan, t..."
3,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000004,20,"Saat ini, banyak kafe-kafe baru yang berlomba ...","Karena ini tempat baru, maka ada antrian panja...","Karena ini tempat baru , maka ada antrian panj...","[Karena, ini, tempat, baru, ,, maka, ada, antr..."
4,20230808_newsdata_Korea_000400,C:/Users/USER/Desktop/news data/20230808_data_...,"'Tangga Menuju Surga,' Konsep Kafe Baru Di Jeo...",Newspaper,Korea,2019-08-25,2023-08-08,20230808_newsdata_Korea_000400_sen000005,46,"Saat ini, banyak kafe-kafe baru yang berlomba ...",Dengan pemandangan menakjubkan dari lampu yang...,Dengan pemandangan menakjubkan dari lampu yang...,"[Dengan, pemandangan, menakjubkan, dari, lampu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103423,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000072,18,Trashbusters tidak putus asa saat memandang tu...,Era yang memungkinkan apa saja dapat diantar m...,Era yang memungkinkan apa saja dapat diantar m...,"[Era, yang, memungkinkan, apa, saja, dapat, di..."
103424,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000073,23,Trashbusters tidak putus asa saat memandang tu...,Meski penggunaan kantong plastik sudah dilaran...,Meski penggunaan kantong plastik sudah dilaran...,"[Meski, penggunaan, kantong, plastik, sudah, d..."
103425,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000074,10,Trashbusters tidak putus asa saat memandang tu...,Hal-hal itu mungkin dapat memicu tawa sinis.,Hal - hal itu mungkin dapat memicu tawa sinis .,"[Hal, -, hal, itu, mungkin, dapat, memicu, taw..."
103426,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000075,9,Trashbusters tidak putus asa saat memandang tu...,"Meskipun demikian, mari kita mengucapkan sloga...","Meskipun demikian , mari kita mengucapkan slog...","[Meskipun, demikian, ,, mari, kita, mengucapka..."


In [25]:
unique_df = unique_df.to_csv('20230926_원천데이터(기사).csv', index = False, encoding = 'utf-8-sig')

In [35]:
df = pd.read_csv('20230926_원천데이터(기사).csv')
df.tail()

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
103423,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000072,18,Trashbusters tidak putus asa saat memandang tu...,Era yang memungkinkan apa saja dapat diantar m...,Era yang memungkinkan apa saja dapat diantar m...,"['Era', 'yang', 'memungkinkan', 'apa', 'saja',..."
103424,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000073,23,Trashbusters tidak putus asa saat memandang tu...,Meski penggunaan kantong plastik sudah dilaran...,Meski penggunaan kantong plastik sudah dilaran...,"['Meski', 'penggunaan', 'kantong', 'plastik', ..."
103425,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000074,10,Trashbusters tidak putus asa saat memandang tu...,Hal-hal itu mungkin dapat memicu tawa sinis.,Hal - hal itu mungkin dapat memicu tawa sinis .,"['Hal', '-', 'hal', 'itu', 'mungkin', 'dapat',..."
103426,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000075,9,Trashbusters tidak putus asa saat memandang tu...,"Meskipun demikian, mari kita mengucapkan sloga...","Meskipun demikian , mari kita mengucapkan slog...","['Meskipun', 'demikian', ',', 'mari', 'kita', ..."
103427,20230920_newsdata_Koreana2_000270,C:/Users/USER/Desktop/news data/20230920_data/...,Trashbusters: Gerakan Lingkungan sebagai Budaya,Newspaper,Koreana2,2023-04-01,2023-09-20,20230920_newsdata_Koreana2_000270_sen000076,33,Trashbusters tidak putus asa saat memandang tu...,"Kemudian, mari kita mengingatkan diri kita bah...","Kemudian , mari kita mengingatkan diri kita ba...","['Kemudian', ',', 'mari', 'kita', 'mengingatka..."


In [36]:
df = df.iloc[50001:100000]
df


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
50001,20230920_newsdata_Koreana1_000260,C:/Users/USER/Desktop/news data/20230920_data/...,MI DALAM MANGKUK,Newspaper,Koreana1,2015-07-01,2023-09-20,20230920_newsdata_Koreana1_000260_sen000116,17,"Ya, betul. Sekarang meremas waktu. Suatu ketik...","Meringkuk di depan televisi, aku meremas adona...","Meringkuk di depan televisi , aku meremas adon...","['Meringkuk', 'di', 'depan', 'televisi', ',', ..."
50002,20230920_newsdata_Koreana1_000260,C:/Users/USER/Desktop/news data/20230920_data/...,MI DALAM MANGKUK,Newspaper,Koreana1,2015-07-01,2023-09-20,20230920_newsdata_Koreana1_000260_sen000117,29,"Ya, betul. Sekarang meremas waktu. Suatu ketik...","Memasak sendiri, aku tidak punya penggilingan ...","Memasak sendiri , aku tidak punya penggilingan...","['Memasak', 'sendiri', ',', 'aku', 'tidak', 'p..."
50003,20230920_newsdata_Koreana1_000260,C:/Users/USER/Desktop/news data/20230920_data/...,MI DALAM MANGKUK,Newspaper,Koreana1,2015-07-01,2023-09-20,20230920_newsdata_Koreana1_000260_sen000118,13,"Ya, betul. Sekarang meremas waktu. Suatu ketik...","Ketika kukeluarkan dari kulkas, gumpalan adona...","Ketika kukeluarkan dari kulkas , gumpalan adon...","['Ketika', 'kukeluarkan', 'dari', 'kulkas', ',..."
50004,20230920_newsdata_Koreana1_000260,C:/Users/USER/Desktop/news data/20230920_data/...,MI DALAM MANGKUK,Newspaper,Koreana1,2015-07-01,2023-09-20,20230920_newsdata_Koreana1_000260_sen000120,11,"Ya, betul. Sekarang meremas waktu. Suatu ketik...",Aku rasa gumpalan adonan harus kulemparkan ke ...,Aku rasa gumpalan adonan harus kulemparkan ke ...,"['Aku', 'rasa', 'gumpalan', 'adonan', 'harus',..."
50005,20230920_newsdata_Koreana1_000260,C:/Users/USER/Desktop/news data/20230920_data/...,MI DALAM MANGKUK,Newspaper,Koreana1,2015-07-01,2023-09-20,20230920_newsdata_Koreana1_000260_sen000121,34,"Ya, betul. Sekarang meremas waktu. Suatu ketik...",Seperti nasib batu yang menggelinding ke sana ...,Seperti nasib batu yang menggelinding ke sana ...,"['Seperti', 'nasib', 'batu', 'yang', 'menggeli..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,20230920_newsdata_Koreana2_000227,C:/Users/USER/Desktop/news data/20230920_data/...,"Andong, Semangat Korea",Newspaper,Koreana2,2022-10-01,2023-09-20,20230920_newsdata_Koreana2_000227_sen000080,20,Andong memadukan masa lalu dan masa kini dalam...,"Museum Topeng Hahoe, yang terletak agak jauh s...","Museum Topeng Hahoe , yang terletak agak jauh ...","['Museum', 'Topeng', 'Hahoe', ',', 'yang', 'te..."
99996,20230920_newsdata_Koreana2_000227,C:/Users/USER/Desktop/news data/20230920_data/...,"Andong, Semangat Korea",Newspaper,Koreana2,2022-10-01,2023-09-20,20230920_newsdata_Koreana2_000227_sen000081,28,Andong memadukan masa lalu dan masa kini dalam...,Itu dihiasi dengan topeng Hahoe yang tersenyum...,Itu dihiasi dengan topeng Hahoe yang tersenyum...,"['Itu', 'dihiasi', 'dengan', 'topeng', 'Hahoe'..."
99997,20230920_newsdata_Koreana2_000227,C:/Users/USER/Desktop/news data/20230920_data/...,"Andong, Semangat Korea",Newspaper,Koreana2,2022-10-01,2023-09-20,20230920_newsdata_Koreana2_000227_sen000082,28,Andong memadukan masa lalu dan masa kini dalam...,"Bergerak daripada berdiri diam, menemukan ceri...","Bergerak daripada berdiri diam , menemukan cer...","['Bergerak', 'daripada', 'berdiri', 'diam', ',..."
99998,20230920_newsdata_Koreana2_000227,C:/Users/USER/Desktop/news data/20230920_data/...,"Andong, Semangat Korea",Newspaper,Koreana2,2022-10-01,2023-09-20,20230920_newsdata_Koreana2_000227_sen000083,47,Andong memadukan masa lalu dan masa kini dalam...,"Di kota, toko-toko tua yang diwariskan dari ge...","Di kota , toko - toko tua yang diwariskan dari...","['Di', 'kota', ',', 'toko', '-', 'toko', 'tua'..."


In [33]:
df = df.to_csv('20230926_원천데이터(기사)_9.csv',index = False, encoding = 'utf-8-sig')

<br>
<br>

# 완료된 데이터프레임 pickle로 변환

In [55]:

df_final.to_pickle('20230911_원천데이터(기사).pickle')

In [12]:
df_final=pd.read_pickle('20230906_원천데이터(기사).pickle')

In [13]:
df_final.sort_values('Sen_ID', inplace=True)
df_final.reset_index(drop=True, inplace=True)

<br>
<br>

## 조각 나누기

In [56]:
chunk_size = 50000

# 데이터프레임을 작은 조각으로 나눈다
chunks = [df_final[i:i+chunk_size] for i in range(0, len(df_final), chunk_size)]

# 작은 조각들을 순회하면서 CSV 파일로 저장
for idx, chunk in enumerate(chunks):
    filename = f"202300911_원천데이터(기사)_{idx+1}.csv"
    chunk.to_csv('../'+filename,encoding='utf-8-sig' ,index=False)

print("CSV 파일로 저장이 완료되었습니다.")

CSV 파일로 저장이 완료되었습니다.


In [51]:
'''
원시데이터/원천데이터/날짜_원천데이터(기사)/ 폴더 안에 최종 csv파일 만들기

import os
import datetime

now = datetime.datetime.now()
now_format = now.strftime("%Y%m%d")

os.mkdir(f"../원천데이터/{now_format}_원천데이터(기사)")

chunk_size = 50000

# 데이터프레임을 작은 조각으로 나눈다
chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

# 작은 조각들을 순회하면서 CSV 파일로 저장
for idx, chunk in enumerate(chunks):
    filename = f"20230821_원천데이터(기사)_{idx+1}.csv"
    chunk.to_csv(f'../원천데이터/{now_format}_원천데이터(기사)/'+filename,encoding='utf-8-sig' ,index=False)

print("CSV 파일로 저장이 완료되었습니다.")
'''

<br>
<br>
<br>

### Pickle 파일 정리
- 2023mmdd_news_assemble_unique.pickle : 토큰화 완료된 파일
- 20230904_원천데이터(기사).pickle : 위의 파일에서 가공된 데이터 제외한 파일
> unique 붙은것은 토큰화 완료된 것


### csv 파일 정리
- 상위폴더의 2023mmdd_원천데이터(기사)_{idx+1}.csv : 최종본